# Data Preprocessing and Analysis

In [1]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 3.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold

In [3]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
df = pd.read_csv('/content/BEA 2024 Task Data Extended_new.csv')
df.head()

,ItemNum,ItemStem_Text,Answer__A,Answer__B,Answer__C,Answer__D,Answer__E,Answer__F,Answer__G,Answer__H,Answer__I,Answer__J,Answer_Key,Answer_Text,ItemType,EXAM,Difficulty,Response_Time
0,622,A 27-year-old man comes to the emergency depar...,Choristoma,Ciliary epithelium,Endothelial cells,Gastric mucosa,Striated muscle,NaN,NaN,NaN,NaN,NaN,D,Gastric mucosa,Text,STEP 1,0.38,123.96
1,440,"A 39-year-old woman, gravida 2, para 2, comes ...",Future fertility plans,Hypertension,Obesity,Patient age,Stage of disease,NaN,NaN,NaN,NaN,NaN,E,Stage of disease,Text,STEP 3,0.50,77.53
2,12,An 18-year-old primigravid woman comes for her...,Repeat measurement of MSAFP concentration,"Triple screening for MSAFP, serum ÃŽÂ²-hCG, an...",Ultrasonography,Amniocentesis for measurement of ÃŽÂ±-fetoprot...,Amniocentesis for chromosomal analysis,NaN,NaN,NaN,NaN,NaN,C,Ultrasonography,Text,STEP 2,0.58,65.15
3,5,"A 26-year-old man, who is admitted to the hosp...",There is no evidence of a familial coagulation...,Half of their daughters will have a clinically...,The sons of their daughters will be at risk fo...,Their sons will be at risk for a clinically ev...,Their sons and daughters will be at risk for a...,NaN,NaN,NaN,NaN,NaN,C,The sons of their daughters will be at risk fo...,Text,STEP 3,0.56,105.24
4,221,A 60-year-old man had a total thyroidectomy an...,Babinski sign present bilaterally,Chvostek sign,Deviation of the tongue to the left side,A drooping left shoulder,Hyporeflexia,NaN,NaN,NaN,NaN,NaN,B,Chvostek sign,Text,STEP 3,0.22,57.02


In [5]:
df.fillna(0, inplace=True)
df.head(2)

,ItemNum,ItemStem_Text,Answer__A,Answer__B,Answer__C,Answer__D,Answer__E,Answer__F,Answer__G,Answer__H,Answer__I,Answer__J,Answer_Key,Answer_Text,ItemType,EXAM,Difficulty,Response_Time
0,622,A 27-year-old man comes to the emergency depar...,Choristoma,Ciliary epithelium,Endothelial cells,Gastric mucosa,Striated muscle,0,0,0,0,0,D,Gastric mucosa,Text,STEP 1,0.38,123.96
1,440,"A 39-year-old woman, gravida 2, para 2, comes ...",Future fertility plans,Hypertension,Obesity,Patient age,Stage of disease,0,0,0,0,0,E,Stage of disease,Text,STEP 3,0.50,77.53


In [6]:
cols = df.columns
cols

Index(['ItemNum', 'ItemStem_Text', 'Answer__A', 'Answer__B', 'Answer__C',
       'Answer__D', 'Answer__E', 'Answer__F', 'Answer__G', 'Answer__H',
       'Answer__I', 'Answer__J', 'Answer_Key', 'Answer_Text', 'ItemType',
       'EXAM', 'Difficulty', 'Response_Time'],
      dtype='object')

In [7]:
df.ItemType.value_counts()

ItemType
Text    595
PIX      72
Name: count, dtype: int64

In [8]:
df.EXAM.value_counts()

EXAM
STEP 1    273
STEP 2    220
STEP 3    174
Name: count, dtype: int64

In [9]:
X_cols = ['ItemNum', 'ItemStem_Text', 'Answer__A', 'Answer__B', 'Answer__C',
           'Answer__D', 'Answer__E', 'Answer__F', 'Answer__G', 'Answer__H',
           'Answer__I', 'Answer__J', 'Answer_Key', 'Answer_Text']
y_1 = df['Difficulty']
y_2 = df['Response_Time']

In [10]:
df[df['ItemNum'] == 285]
df[X_cols] = df[X_cols].astype('str')

# Dividing data into train and test set

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df[X_cols], df.index, test_size=0.2, random_state=1)

y_train1, y_test1 = y_1[y_train], y_1[y_test]
y_train2, y_test2 = y_2[y_train], y_2[y_test]

X_train1, X_test1 = X_train, X_test
X_train2, X_test2 = X_train, X_test

#  Vectorizing input using CBOW

In [12]:
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [word for word in tokens if word.isalnum() and word not in stop_words]
    return tokens

def vectorizer(data):
    all_sentences = []
    for index, row in tqdm(data.iterrows()):
        all_sentences.append(preprocess_text(row['ItemStem_Text']))
        for option in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']:
            if row['Answer__' + option] != 0:
                all_sentences.append(preprocess_text(row['Answer__' + option]))
        all_sentences.append(preprocess_text(row['Answer_Text']))

    # Train Word2Vec model
    model = Word2Vec(all_sentences, min_count=1, vector_size=100, window=5, sg=0)

    data_vec = []
    for index, row in tqdm(data.iterrows()):
        vectors = []
        vectors.append(get_sentence_vector(model, row['ItemStem_Text']))
        for option in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']:
            if row['Answer__' + option] != 0:
                vectors.append(get_sentence_vector(model, row['Answer__' + option]))
        vectors.append(get_sentence_vector(model, row['Answer_Text']))
        data_vec.append(np.mean(vectors, axis=0))
    return np.array(data_vec)


def get_sentence_vector(model, sentence):
    words = preprocess_text(sentence)
    vectors = [model.wv[word] for word in words if word in model.wv]
    return np.mean(vectors, axis=0) if vectors else np.zeros(model.vector_size)

In [13]:
X_vec = vectorizer(df[X_cols])
X_vec.shape

667it [00:01, 371.54it/s]
667it [00:02, 279.26it/s]


(667, 100)

In [14]:
scaler = StandardScaler()
X_vec_scaled = scaler.fit_transform(X_vec)

# Implement K-Fold Cross-Validation

In [15]:
kfold = KFold(n_splits=5, shuffle=True, random_state=1)
y1_predict_full = np.zeros(len(X_vec_scaled))
y2_predict_full = np.zeros(len(X_vec_scaled))
rmse_scores_1 = []

# CBOW Model

In [27]:
import tensorflow as tf
from tensorflow.keras import layers

def build_CBOW_model():
    model = tf.keras.Sequential([
        tf.keras.layers.InputLayer(input_shape=(100,)),  # Adjusted for CBOW vector size
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.5),                            # Added dropout for regularization
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),                            # Added another dropout layer
        layers.Dense(64, activation='relu'),
        layers.Dense(32, activation='relu'),
        layers.Dense(1)
    ])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss=tf.keras.losses.MeanSquaredError(),
                  metrics=[tf.keras.metrics.RootMeanSquaredError()])

    return model

# CBOW Model - Predicting Difficulty

In [28]:
for train_index, test_index in kfold.split(X_vec_scaled):
    X_train, X_test = X_vec_scaled[train_index], X_vec_scaled[test_index]
    y_train1, y_test1 = y_1[train_index], y_1[test_index]
    y_train2, y_test2 = y_2[train_index], y_2[test_index]

    dl_model_1 = build_CBOW_model()
    dl_model_1.fit(X_train, y_train1, epochs=100, verbose=1)

    y1_predict = dl_model_1.predict(X_test)
    y1_predict_full[test_index] = y1_predict.flatten()
    mse = mean_squared_error(y_test1, y1_predict)
    rmse = np.sqrt(mse)
    rmse_scores_1.append(rmse)

Epoch 1/100
17/17 [==============================] - 1s 3ms/step - loss: 0.7519 - root_mean_squared_error: 0.8671
Epoch 2/100
17/17 [==============================] - 0s 3ms/step - loss: 0.2284 - root_mean_squared_error: 0.4780
Epoch 3/100
17/17 [==============================] - 0s 3ms/step - loss: 0.2387 - root_mean_squared_error: 0.4886
Epoch 4/100
17/17 [==============================] - 0s 4ms/step - loss: 0.1778 - root_mean_squared_error: 0.4217
Epoch 5/100
17/17 [==============================] - 0s 3ms/step - loss: 0.1713 - root_mean_squared_error: 0.4138
Epoch 6/100
17/17 [==============================] - 0s 3ms/step - loss: 0.1676 - root_mean_squared_error: 0.4094
Epoch 7/100
17/17 [==============================] - 0s 3ms/step - loss: 0.1424 - root_mean_squared_error: 0.3773
Epoch 8/100
17/17 [==============================] - 0s 3ms/step - loss: 0.1409 - root_mean_squared_error: 0.3754
Epoch 9/100
17/17 [==============================] - 0s 3ms/step - loss: 0.1429 - root_m

In [29]:
y1_predict = dl_model_1.predict(X_vec)  # Predict on all data

for i, prediction in enumerate(y1_predict):
    print(f"Entry {i+1}: Predicted Difficulty = {prediction[0]:.4f}")  # Display with 4 decimal places

21/21 [==============================] - 0s 2ms/step
Entry 1: Predicted Difficulty = 0.6087
Entry 2: Predicted Difficulty = 0.5767
Entry 3: Predicted Difficulty = 0.5833
Entry 4: Predicted Difficulty = 0.6032
Entry 5: Predicted Difficulty = 0.6019
Entry 6: Predicted Difficulty = 0.6072
Entry 7: Predicted Difficulty = 0.6165
Entry 8: Predicted Difficulty = 0.5996
Entry 9: Predicted Difficulty = 0.6372
Entry 10: Predicted Difficulty = 0.6173
Entry 11: Predicted Difficulty = 0.6083
Entry 12: Predicted Difficulty = 0.6191
Entry 13: Predicted Difficulty = 0.6298
Entry 14: Predicted Difficulty = 0.5847
Entry 15: Predicted Difficulty = 0.6286
Entry 16: Predicted Difficulty = 0.6111
Entry 17: Predicted Difficulty = 0.6060
Entry 18: Predicted Difficulty = 0.6253
Entry 19: Predicted Difficulty = 0.5941
Entry 20: Predicted Difficulty = 0.6243
Entry 21: Predicted Difficulty = 0.6026
Entry 22: Predicted Difficulty = 0.5691
Entry 23: Predicted Difficulty = 0.6071
Entry 24: Predicted Difficulty = 0.6

In [30]:
print("RMSE for Difficulty with K-Fold Cross-Validation:", np.mean(rmse_scores_1))
print("_________________________________________________________________")
dl_model_1.summary()

RMSE for Difficulty with K-Fold Cross-Validation: 0.3401520234369499
_________________________________________________________________
Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_60 (Dense)            (None, 256)               25856     
                                                                 
 dropout_12 (Dropout)        (None, 256)               0         
                                                                 
 dense_61 (Dense)            (None, 128)               32896     
                                                                 
 dropout_13 (Dropout)        (None, 128)               0         
                                                                 
 dense_62 (Dense)            (None, 64)                8256      
                                                                 
 dense_63 (Dense)            (None, 32)           

# CBOW Model - Predicting Response Time

In [31]:
rmse_scores_2 = []

for train_index, test_index in kfold.split(X_vec_scaled):
    X_train, X_test = X_vec_scaled[train_index], X_vec_scaled[test_index]
    y_train2, y_test2 = y_2[train_index], y_2[test_index]

    dl_model_2 = build_CBOW_model()
    dl_model_2.fit(X_train, y_train2, epochs=100, verbose=1)

    y2_predict = dl_model_2.predict(X_test)
    y2_predict_full[test_index] = y2_predict.flatten()
    mse = mean_squared_error(y_test2, y2_predict)
    rmse = np.sqrt(mse)
    rmse_scores_2.append(rmse)

Epoch 1/100
17/17 [==============================] - 1s 3ms/step - loss: 7742.6577 - root_mean_squared_error: 87.9924
Epoch 2/100
17/17 [==============================] - 0s 3ms/step - loss: 4348.5063 - root_mean_squared_error: 65.9432
Epoch 3/100
17/17 [==============================] - 0s 2ms/step - loss: 3459.0647 - root_mean_squared_error: 58.8138
Epoch 4/100
17/17 [==============================] - 0s 2ms/step - loss: 3255.4905 - root_mean_squared_error: 57.0569
Epoch 5/100
17/17 [==============================] - 0s 5ms/step - loss: 3336.2529 - root_mean_squared_error: 57.7603
Epoch 6/100
17/17 [==============================] - 0s 5ms/step - loss: 3045.9951 - root_mean_squared_error: 55.1905
Epoch 7/100
17/17 [==============================] - 0s 5ms/step - loss: 2866.7705 - root_mean_squared_error: 53.5422
Epoch 8/100
17/17 [==============================] - 0s 4ms/step - loss: 2765.8574 - root_mean_squared_error: 52.5914
Epoch 9/100
17/17 [==============================] - 0s 

In [32]:
y2_predict = dl_model_2.predict(X_vec)
for i, prediction in enumerate(y2_predict):
    print(f"Entry {i+1}: Predicted Difficulty = {prediction[0]:.4f}")

21/21 [==============================] - 0s 2ms/step
Entry 1: Predicted Difficulty = 89.6029
Entry 2: Predicted Difficulty = 89.3546
Entry 3: Predicted Difficulty = 89.2584
Entry 4: Predicted Difficulty = 89.3932
Entry 5: Predicted Difficulty = 89.4642
Entry 6: Predicted Difficulty = 89.4363
Entry 7: Predicted Difficulty = 89.4476
Entry 8: Predicted Difficulty = 89.2930
Entry 9: Predicted Difficulty = 89.1928
Entry 10: Predicted Difficulty = 89.4769
Entry 11: Predicted Difficulty = 89.4631
Entry 12: Predicted Difficulty = 89.4589
Entry 13: Predicted Difficulty = 89.4377
Entry 14: Predicted Difficulty = 89.2021
Entry 15: Predicted Difficulty = 89.3280
Entry 16: Predicted Difficulty = 89.5477
Entry 17: Predicted Difficulty = 89.5969
Entry 18: Predicted Difficulty = 89.5198
Entry 19: Predicted Difficulty = 89.3784
Entry 20: Predicted Difficulty = 89.3296
Entry 21: Predicted Difficulty = 89.3307
Entry 22: Predicted Difficulty = 89.3241
Entry 23: Predicted Difficulty = 89.4823
Entry 24: Pre

In [33]:
print("RMSE for Response Time with K-Fold Cross-Validation:", np.mean(rmse_scores_2))
print('_________________________________________________________________')
dl_model_2.summary()

RMSE for Response Time with K-Fold Cross-Validation: 30.65716882310266
_________________________________________________________________
Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_85 (Dense)            (None, 256)               25856     
                                                                 
 dropout_22 (Dropout)        (None, 256)               0         
                                                                 
 dense_86 (Dense)            (None, 128)               32896     
                                                                 
 dropout_23 (Dropout)        (None, 128)               0         
                                                                 
 dense_87 (Dense)            (None, 64)                8256      
                                                                 
 dense_88 (Dense)            (None, 32)         

# Add predicted values

In [34]:
df['Predicted_Difficulty'] = y1_predict_full
df['Predicted_Response_Time'] = y2_predict_full
df


,ItemNum,ItemStem_Text,Answer__A,Answer__B,Answer__C,Answer__D,Answer__E,Answer__F,Answer__G,Answer__H,Answer__I,Answer__J,Answer_Key,Answer_Text,ItemType,EXAM,Difficulty,Response_Time,Predicted_Difficulty,Predicted_Response_Time
0,622,A 27-year-old man comes to the emergency depar...,Choristoma,Ciliary epithelium,Endothelial cells,Gastric mucosa,Striated muscle,0,0,0,0,0,D,Gastric mucosa,Text,STEP 1,0.38,123.96,0.574628,96.928642
1,440,"A 39-year-old woman, gravida 2, para 2, comes ...",Future fertility plans,Hypertension,Obesity,Patient age,Stage of disease,0,0,0,0,0,E,Stage of disease,Text,STEP 3,0.50,77.53,0.506774,94.381477
2,12,An 18-year-old primigravid woman comes for her...,Repeat measurement of MSAFP concentration,"Triple screening for MSAFP, serum ÃŽÂ²-hCG, an...",Ultrasonography,Amniocentesis for measurement of ÃŽÂ±-fetoprot...,Amniocentesis for chromosomal analysis,0,0,0,0,0,C,Ultrasonography,Text,STEP 2,0.58,65.15,0.457064,91.041260
3,5,"A 26-year-old man, who is admitted to the hosp...",There is no evidence of a familial coagulation...,Half of their daughters will have a clinically...,The sons of their daughters will be at risk fo...,Their sons will be at risk for a clinically ev...,Their sons and daughters will be at risk for a...,0,0,0,0,0,C,The sons of their daughters will be at risk fo...,Text,STEP 3,0.56,105.24,0.440276,77.338928
4,221,A 60-year-old man had a total thyroidectomy an...,Babinski sign present bilaterally,Chvostek sign,Deviation of the tongue to the left side,A drooping left shoulder,Hyporeflexia,0,0,0,0,0,B,Chvostek sign,Text,STEP 3,0.22,57.02,0.538763,84.944664
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
662,612,A 45-year-old man is brought to the clinic by ...,Atrophy,Decreased visual acuity,Loss of proprioception,Presence of palmomental reflex,Ptosis,0,0,0,0,0,A,Atrophy,Text,STEP 1,0.22,95.17,0.465629,84.457733
663,315,A 64-year-old man with non-Hodgkin lymphoma co...,Bleomycin,Cyclophosphamide,Cytarabine,Doxorubicin,Fluorouracil,Methotrexate,Vincristine,0,0,0,G,Vincristine,Text,STEP 1,0.14,42.72,0.494240,68.773827
664,509,A 9-month-old boy is brought to the office by ...,Abdominal ultrasonography,Antiâ€“Saccharomyces cerevisiae antibody testing,Nitroblue tetrazolium testing,Rectal swab for group B streptococcus,Skeletal survey,Stool culture for Salmonella enteritidis,0,0,0,0,E,Skeletal survey,Text,STEP 2,0.94,122.73,0.456406,78.423180
665,550,A 32-year-old man comes to the office because ...,Collagen,Double-stranded DNA,Nucleolar protein,Phospholipid,Proteins in neutrophil cytoplasm,0,0,0,0,0,A,Collagen,Text,STEP 1,0.30,86.13,0.467684,83.142563


In [24]:
df.to_csv("Predicted_Difficulty_Response_time_CBOW.csv")